In [359]:
import pandas as pd
import numpy as np

In [360]:
df = pd.read_csv("Churn_Modelling.csv")

In [361]:
df.drop(columns=['RowNumber' , 'CustomerId' , 'Surname'],inplace=True)

In [362]:
categorical_features = df.select_dtypes(include='O').columns

In [363]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

# Applying OHE in Geography and Ordinal Encoder on Gender Feature

Female = 0

Male = 1

In [364]:
df

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              1   
9998          772   Germany    Male   42       3   75075.31              2   
9999          792    France  Female   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0             1               1        101348.88       1  
1             0               1        112542.58       0  
2             1               0        113931.57       1  
3             0               0         93826.63       0  
4             1               1         79084.10       0  
...         ...             ...              ...     ...  
9995          1               0         96270.64       0  
9996          1               1        101699.77       0  
9997          0               1         42085.58       1  
9998          1               0         92888.52       1  
9999          1               0         38190.78       0  

[10000 rows x 11 columns]

In [365]:
from sklearn.preprocessing import StandardScaler , OneHotEncoder , OrdinalEncoder

In [366]:
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer

In [367]:
cols = df.columns

In [368]:
X = df.drop(columns='Exited')
y = df['Exited']

In [369]:
preprocessing = ColumnTransformer(
    transformers=[
        ('Gender_Ordinal_Encoder' , OrdinalEncoder() , ['Gender']),
        ('Geography_OHE' , OneHotEncoder(handle_unknown='ignore' , sparse_output=False) , ['Geography'])
    ],remainder=StandardScaler()
)

In [370]:
df_data = preprocessing.fit_transform(X)

In [371]:
preprocessing

ColumnTransformer(remainder=StandardScaler(),
                  transformers=[('Gender_Ordinal_Encoder', OrdinalEncoder(),
                                 ['Gender']),
                                ('Geography_OHE',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Geography'])])

In [372]:
preprocessing.get_feature_names_out()

array(['Gender_Ordinal_Encoder__Gender',
       'Geography_OHE__Geography_France',
       'Geography_OHE__Geography_Germany',
       'Geography_OHE__Geography_Spain', 'remainder__CreditScore',
       'remainder__Age', 'remainder__Tenure', 'remainder__Balance',
       'remainder__NumOfProducts', 'remainder__HasCrCard',
       'remainder__IsActiveMember', 'remainder__EstimatedSalary'],
      dtype=object)

In [373]:
X_preprocessed = pd.DataFrame(df_data , columns=preprocessing.get_feature_names_out())

In [374]:
df.shape

(10000, 11)

In [375]:
X_preprocessed

Gender_Ordinal_Encoder__Gender  Geography_OHE__Geography_France  \
0                                0.0                              1.0   
1                                0.0                              0.0   
2                                0.0                              1.0   
3                                0.0                              1.0   
4                                0.0                              0.0   
...                              ...                              ...   
9995                             1.0                              1.0   
9996                             1.0                              1.0   
9997                             0.0                              1.0   
9998                             1.0                              0.0   
9999                             0.0                              1.0   

      Geography_OHE__Geography_Germany  Geography_OHE__Geography_Spain  \
0                                  0.0                             0.0   
1                                  0.0                             1.0   
2                                  0.0                             0.0   
3                                  0.0                             0.0   
4                                  0.0                             1.0   
...                                ...                             ...   
9995                               0.0                             0.0   
9996                               0.0                             0.0   
9997                               0.0                             0.0   
9998                               1.0                             0.0   
9999                               0.0                             0.0   

      remainder__CreditScore  remainder__Age  remainder__Tenure  \
0                  -0.326221        0.293517          -1.041760   
1                  -0.440036        0.198164          -1.387538   
2                  -1.536794        0.293517           1.032908   
3                   0.501521        0.007457          -1.387538   
4                   2.063884        0.388871          -1.041760   
...                      ...             ...                ...   
9995                1.246488        0.007457          -0.004426   
9996               -1.391939       -0.373958           1.724464   
9997                0.604988       -0.278604           0.687130   
9998                1.256835        0.293517          -0.695982   
9999                1.463771       -1.041433          -0.350204   

      remainder__Balance  remainder__NumOfProducts  remainder__HasCrCard  \
0              -1.225848                 -0.911583              0.646092   
1               0.117350                 -0.911583             -1.547768   
2               1.333053                  2.527057              0.646092   
3              -1.225848                  0.807737             -1.547768   
4               0.785728                 -0.911583              0.646092   
...                  ...                       ...                   ...   
9995           -1.225848                  0.807737              0.646092   
9996           -0.306379                 -0.911583              0.646092   
9997           -1.225848                 -0.911583             -1.547768   
9998           -0.022608                  0.807737              0.646092   
9999            0.859965                 -0.911583              0.646092   

      remainder__IsActiveMember  remainder__EstimatedSalary  
0                      0.970243                    0.021886  
1                      0.970243                    0.216534  
2                     -1.030670                    0.240687  
3                     -1.030670                   -0.108918  
4                      0.970243                   -0.365276  
...                         ...                         ...  
9995                  -1.030670                   -0.066419  
9996                   0.9

In [376]:
import pickle

with open('preprocessed.pkl' , 'wb') as file:
    pickle.dump(preprocessing,file)

# Training The Model

In [377]:
cols = X_preprocessed.columns

In [378]:
cols

Index(['Gender_Ordinal_Encoder__Gender', 'Geography_OHE__Geography_France',
       'Geography_OHE__Geography_Germany', 'Geography_OHE__Geography_Spain',
       'remainder__CreditScore', 'remainder__Age', 'remainder__Tenure',
       'remainder__Balance', 'remainder__NumOfProducts',
       'remainder__HasCrCard', 'remainder__IsActiveMember',
       'remainder__EstimatedSalary'],
      dtype='object')

In [379]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X_preprocessed , y , test_size=0.2 , random_state=42)

In [380]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [381]:
# with open('scalar.pkl' , 'wb') as file:
#     pickle.dump(scalar , file)

# Applying RandomForest on this dataset

In [382]:
from sklearn.ensemble import RandomForestClassifier

In [383]:
RFC = RandomForestClassifier(n_estimators=100 , oob_score=True)
RFC.fit(X_train , y_train)

RandomForestClassifier(oob_score=True)

In [384]:
y_pred = RFC.predict(X_test)

In [385]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.8645

# Applying XGBoost ALgo

In [386]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train , y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [387]:
y_pred = xgb.predict(X_test)

In [388]:
accuracy_score(y_test , y_pred)

0.858

# Applying ANN

In [389]:
import tensorflow
from tensorflow import keras 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [390]:
X_train.shape

(8000, 12)

In [391]:
from tensorflow.keras.regularizers import l2

In [392]:
model = Sequential()

model.add(Dense(64 , activation='relu' , input_dim = X_train.shape[1] , kernel_regularizer=l2(0.001) , kernel_initializer= 'he_uniform'))

model.add(Dense(32 , activation='relu' , kernel_regularizer=l2(0.001) , kernel_initializer='he_uniform'))

model.add(Dense(32 , activation='relu' , kernel_regularizer=l2(0.001) , kernel_initializer='he_uniform'))

model.add(Dense(1 , activation='sigmoid'))

C:\Users\LeoML\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [393]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,001 (15.63 KB)

 Trainable params: 4,001 (15.63 KB)

 Non-trainable params: 0 (0.00 B)

In [394]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [395]:
model.compile(loss=loss , optimizer=opt , metrics=['accuracy'])

## Designing TensorBoard

In [396]:
import datetime

### Tensorboard is used to visualize all the logs that we have while training the model

In [397]:
from tensorflow.keras.callbacks import TensorBoard

log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir = log_dir , histogram_freq = 1)

In [398]:
from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 0.0001,
    patience = 20 ,
    mode = 'auto',
    baseline = None
)

In [399]:
history = model.fit(X_train , y_train , epochs=100 ,callbacks=[callback , tensorflow_callback], validation_data=(X_test , y_test))

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8265 - loss: 0.5508 - val_accuracy: 0.8460 - val_loss: 0.4380
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8503 - loss: 0.4130 - val_accuracy: 0.8495 - val_loss: 0.3896
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8499 - loss: 0.3920 - val_accuracy: 0.8505 - val_loss: 0.3882
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8495 - loss: 0.3889 - val_accuracy: 0.8560 - val_loss: 0.3992
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8554 - loss: 0.3781 - val_accuracy: 0.8560 - val_loss: 0.3693
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8536 - loss: 0.3745 - val_accuracy: 0.8475 - val_loss: 0.3812
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8546 - loss: 0.3739 - val_accuracy: 0.8545 - val_loss: 0.3573
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8577 - loss: 0.3665 - val_accu

In [400]:
model.save('model.keras')

In [401]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [402]:
%tensorboard --logdir log/fit

Reusing TensorBoard on port 6006 (pid 21468), started 1 day, 1:47:13 ago. (Use '!kill 21468' to kill it.)